# Yolo inference with heat map generation
## Imports

In [ ]:
import os
import cv2
import random
import numpy as np
from datetime import datetime
from CellDetector import CellDetector
import matplotlib.pyplot as plt
%matplotlib inline

## Load model

In [ ]:
cell_detector = CellDetector("02_23_all_data_ft.pt")

## Load data

In [ ]:
DEATH_TYPE = "Apoptosis"
CELL_TYPE = "HT29"
base_dir = "../Test Data/" + DEATH_TYPE + "/"
images_dir = base_dir + "Phase/"
green_dir = base_dir + "Green/"
images = os.listdir(images_dir)
len_images = len(images)

In [ ]:
image_id = random.randrange(len_images)
print(image_id)
source = images_dir + images[image_id]
source_green = green_dir + images[image_id]
print(source)

img = cv2.imread(source, cv2.IMREAD_UNCHANGED)

img_green = cv2.imread(source_green, cv2.IMREAD_UNCHANGED)
img_green = cv2.convertScaleAbs(img_green, alpha=(255.0/65535.0))
img_green = cv2.addWeighted(img_green, 8.7, np.zeros(img_green.shape, img_green.dtype), 0, 0.0)

## Predict

In [ ]:
apoptosis_image, necroptosis_image, background_image, big_image, results = cell_detector.predict_with_heatmap(img, 0.2, 0.4, desired_coverage = 10, withImage = True, withResults = True, device=0)

## Visualize

### Heat map

In [ ]:
amax = apoptosis_image.max()
amin = apoptosis_image.min()
print("Apoptosis")
print(amax)
print(amin)
nmax = necroptosis_image.max()
nmin = necroptosis_image.min()
print("Necroptosis")
print(nmax)
print(nmin)
bmax = background_image.max()
bmin = background_image.min()
print("Background")
print(bmax)
print(bmin)

In [ ]:
fig, ((ax1),(ax2), (ax3)) = plt.subplots(nrows=1,ncols=3,figsize=(16,8))
fig.suptitle(str(images[image_id]), fontsize=16)

ax1.imshow(apoptosis_image, vmin=0.0, vmax=1.0, cmap='bone')
ax1.set_title("Apoptosis")

ax2.imshow(necroptosis_image, vmin=0.0, vmax=1.0, cmap='bone')
ax2.set_title("Necroptosis")

ax3.imshow(background_image, vmin=0.0, vmax=1.0, cmap='bone')
ax3.set_title("Background")

In [ ]:
IF_GREEN = True

if IF_GREEN:
    image_boxed, image_green_boxed = cell_detector.box_image(big_image, results, image_green = img_green, prepare_type = 0)
else:
    image_boxed = cell_detector.box_image(big_image, results)

In [ ]:
fig, ((ax1, ax2),(ax3, ax4)) = plt.subplots(nrows=2,ncols=2,figsize=(16,8))
fig.suptitle(str(images[image_id]), fontsize=16)

image_boxed = cv2.cvtColor(image_boxed, cv2.COLOR_BGR2RGB)

image_green_boxed = cv2.cvtColor(image_green_boxed, cv2.COLOR_BGR2RGB)


ax1.imshow(image_boxed)
ax1.set_title("Phase")

ax2.imshow(image_green_boxed)
ax2.set_title("Green")

ax3.imshow(apoptosis_image, vmin=0.0, vmax=1.0, cmap='inferno')
ax3.set_title("Apoptosis")

ax4.imshow(necroptosis_image, vmin=0.0, vmax=1.0, cmap='inferno')
ax4.set_title("Necroptosis")

In [ ]:
plt.imshow(apoptosis_image, vmin=0.0, vmax=1.0, cmap='inferno')

plt.colorbar()
plt.show()